In [1]:
using Distributed; addprocs(5, exeflags="--project")

5-element Vector{Int64}:
 2
 3
 4
 5
 6

In [2]:
# Base functionality
@everywhere begin
    using Plots, Measures, Statistics

    # ODE Solver
    using DifferentialEquations

    # Benchmarking
    using BenchmarkTools

    include("../MixedMealModel.jl")
end

In [15]:
@everywhere function fit_individual(individual)
    glc, ins, trg, nfa, bwg, time = SampleData()
    model, constants, subject = ModelFromData(glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], bwg[individual])
    loss, simulationfunc = LossFunction(model, glc[individual,:], ins[individual,:], trg[individual,:], nfa[individual,:], Int.(time), constants, subject)
    p_init = [1.35e-2, 3.80e-3, 5.82e-1, 0.00045, 0.0713, 208.88, 0.0163, 0.0119]
    p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
    p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]
    println("Initial loss for individual $(individual): $(loss(p_init))")
    res, obj = FitModelLHC(loss, 25, p_lb, p_ub)
    return res, obj
end

In [17]:
BenchmarkResult = pmap(fit_individual, [1, 2, 3, 4, 5]) samples = 100 seconds = 60*60*2;

      From worker 5:	85
      From worker 2:	85
      From worker 4:	85


      From worker 6:	85
      From worker 3:	85


      From worker 3:	Initial loss for individual 4: 643.0113717031269


      From worker 5:	Initial loss for individual 2: 4077.6731794240304


      From worker 6:	Initial loss for individual 3: 2309.195872602529
      From worker 4:	Initial loss for individual 1: 1293.5755081862403


      From worker 2:	Initial loss for individual 5: 936.6104527355869


5-element Vector{Tuple{Vector{Any}, Vector{Float64}}}:
 ([[0.023478801251371834, 0.02085327735740808, 3.62955935332861, 0.0006379870314895386, 0.055142325022782844, 539.7113616057563, 0.011789926633012945, 0.032977159526419456], [0.023468936803888316, 0.02085181201744463, 3.631944232970796, 0.0006377347250468012, 0.05513342818822609, 539.5801016095427, 0.01178878422449186, 0.032968118143668686], [0.02346042976146061, 0.02084943667949514, 3.633226845631409, 0.000636396151730811, 0.0551011740516761, 538.3725489462984, 0.011797998404720254, 0.032923096877535224], [0.023472300574959123, 0.020852285609918326, 3.631097373491749, 0.0006379359559958797, 0.05513958804469486, 539.7590764188313, 0.011787538832627663, 0.03297463057603753], [0.023422718332100842, 0.02084281941684617, 3.639698729736221, 0.0006318432882774011, 0.054993478614004165, 534.2392104351677, 0.011829923125929698, 0.032770020955504374], [0.023456482996633953, 0.020849072521556877, 3.634417380962737, 0.0006362607660335114, 0.0